# Convert Keras Model for Deployment

- Keras Model (.h5) -> Frozen Model (.pb) -> Quantize (.pb) -> Deploy (.elf)

In [2]:
import os
import shutil

In [3]:
import numpy as np

In [4]:
import tensorflow as tf
from tensorflow.python.tools import freeze_graph

In [5]:
print('TensorFlow Version: %s' % tf.__version__)

TensorFlow Version: 1.12.0


***

## Load dataset

In [20]:
# Load dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

***

## Load Keras Model

In [6]:
# model_path = os.path.join('..', 'models', 'yolo-tiny.h5')
model_path = 'lenet.h5'

In [21]:
tf.keras.backend.clear_session()
tf.keras.backend.set_learning_phase(0)

In [8]:
model = tf.keras.models.load_model(model_path)

In [9]:
# model.summary()

In [10]:
batch_size = 100
iter = int(len(x_train)/batch_size)

result_0 = np.zeros(len(x_train))

for i in range(0, iter):
    img = np.expand_dims(x_train[batch_size*i:batch_size*(i+1)], axis=3)/255
    result_0[batch_size*i:batch_size*(i+1)] = np.argmax(model.predict(x=img, batch_size=100), axis=1)

In [11]:
print('Accuracy: %d / %d' % (np.sum(result_0 == y_train), len(x_train)) )

Accuracy: 59824 / 60000


***

## Convert to frozen model

- https://github.com/tensorflow/tensorflow/issues/31331

In [9]:
dst_dir = '_test'

In [10]:
# Delete existing output directory
if os.path.exists(dst_dir):
    shutil.rmtree(dst_dir)
    
tf.saved_model.simple_save(
    tf.keras.backend.get_session(),
    dst_dir,
    inputs={"input": model.inputs[0]},
    outputs={"output": model.outputs[0]})

Instructions for updating:
Pass your op to the equivalent parameter main_op instead.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: _test/saved_model.pb


In [11]:
_ = freeze_graph.freeze_graph(
    None,
    None,
    None,
    None,
    'dense_1/Softmax',
    None,
    None,
    os.path.join(dst_dir, "frozen_model.pb"),
    False,
    "",
    input_saved_model_dir=dst_dir)

INFO:tensorflow:Restoring parameters from _test/variables/variables
INFO:tensorflow:Froze 8 variables.
INFO:tensorflow:Converted 8 variables to const ops.


In [12]:
# if os.path.exists(dst_dir):
#     shutil.rmtree(dst_dir)

# # Save in TF format (saved_model.pb)
# model.save(dst_dir, save_format='tf', include_optimizer=False)

***

## Evaluate frozen model

In [16]:
saved_model_path = os.path.join(dst_dir, 'frozen_model.pb')

In [17]:
input_graph_def = tf.Graph().as_graph_def()
input_graph_def.ParseFromString(tf.gfile.GFile(saved_model_path, "rb").read())

4804016

In [18]:
for node in input_graph_def.node:
    print('%s' % (node.name) )
    
# print('----- Output Node Candidates -----')
# for node in input_graph_def.node:
#     if not tf.get_default_graph().get_tensor_by_name(node.name+':0').consumers():
#         print('%s: %s' % (node.name, tf.get_default_graph().get_tensor_by_name(node.name+':0').shape) )

conv2d_input
conv2d/kernel
conv2d/bias
conv2d/Conv2D/ReadVariableOp
conv2d/Conv2D
conv2d/BiasAdd/ReadVariableOp
conv2d/BiasAdd
conv2d/Relu
conv2d_1/kernel
conv2d_1/bias
conv2d_1/Conv2D/ReadVariableOp
conv2d_1/Conv2D
conv2d_1/BiasAdd/ReadVariableOp
conv2d_1/BiasAdd
conv2d_1/Relu
max_pooling2d/MaxPool
dropout/Identity
flatten/Shape
flatten/strided_slice/stack
flatten/strided_slice/stack_1
flatten/strided_slice/stack_2
flatten/strided_slice
flatten/Reshape/shape/1
flatten/Reshape/shape
flatten/Reshape
dense/kernel
dense/bias
dense/MatMul/ReadVariableOp
dense/MatMul
dense/BiasAdd/ReadVariableOp
dense/BiasAdd
dense/Relu
dropout_1/Identity
dense_1/kernel
dense_1/bias
dense_1/MatMul/ReadVariableOp
dense_1/MatMul
dense_1/BiasAdd/ReadVariableOp
dense_1/BiasAdd
dense_1/Softmax


In [19]:
# Get input tensors
input_tensor = tf.get_default_graph().get_tensor_by_name(model.inputs[0].name)

In [20]:
# Calculate accuracy
output_tensor = tf.get_default_graph().get_tensor_by_name('metrics/acc/ArgMax_1:0')
# output_tensor = tf.get_default_graph().get_tensor_by_name(model.outputs[0].name)

In [ ]:
batch_size = 100
iter = int(len(x_train)/batch_size)

result = np.zeros(len(x_train))

sess = tf.keras.backend.get_session()

for i in range(0, iter):
    img = np.expand_dims(x_train[batch_size*i:batch_size*(i+1)], axis=3)/255

    feed_dict = {input_tensor: img}
    out_0 = sess.run([output_tensor], feed_dict)
    
    result[batch_size*i:batch_size*(i+1)] = out_0[0]

In [ ]:
print('Accuracy: %d / %d' % (np.sum(result == y_train), len(x_train)) )

***

## Evaluate quantized model

In [9]:
quant_model_path = os.path.join('quantize_results', 'quantize_eval_model.pb')

In [12]:
quant_graph_def = tf.Graph().as_graph_def()
quant_graph_def.ParseFromString(tf.gfile.GFile(quant_model_path, "rb").read())

4804326

In [23]:
# Get input/output tensors
input_tensor = tf.get_default_graph().get_tensor_by_name('conv2d_input:0')


KeyError: "The name 'conv2d_input:0' refers to a Tensor which does not exist. The operation, 'conv2d_input', does not exist in the graph."

In [18]:
for node in quant_graph_def.node:
    print('%s' % (node.name) )

conv2d_input
conv2d_input/aquant
dense_1/bias
dense_1/bias/wquant
dense_1/kernel
dense_1/kernel/wquant
dense/bias
dense/bias/wquant
dense/kernel
dense/kernel/wquant
flatten/Reshape/shape/1
flatten/strided_slice/stack_2
flatten/strided_slice/stack_1
flatten/strided_slice/stack
conv2d_1/bias
conv2d_1/bias/wquant
conv2d_1/kernel
conv2d_1/kernel/wquant
conv2d/bias
conv2d/bias/wquant
conv2d/kernel
conv2d/kernel/wquant
conv2d/Conv2D
conv2d/BiasAdd
conv2d/Relu
conv2d/Relu/aquant
conv2d_1/Conv2D
conv2d_1/BiasAdd
conv2d_1/Relu
conv2d_1/Relu/aquant
max_pooling2d/MaxPool
max_pooling2d/MaxPool/aquant
flatten/Shape
flatten/strided_slice
flatten/Reshape/shape
flatten/Reshape
flatten/Reshape/aquant
dense/MatMul
dense/BiasAdd
dense/Relu
dense/Relu/aquant
dense_1/MatMul
dense_1/BiasAdd
dense_1/BiasAdd/aquant
dense_1/Softmax


In [17]:
output_tensor = tf.get_default_graph().get_tensor_by_name('dense_1/Softmax')

ValueError: The name 'dense_1/Softmax' looks like an (invalid) Operation name, not a Tensor. Tensor names must be of the form "<op_name>:<output_index>".

In [ ]:
batch_size = 100
iter = int(len(x_train)/batch_size)

result = np.zeros(len(x_train))

sess = tf.keras.backend.get_session()

for i in range(0, iter):
    img = np.expand_dims(x_train[batch_size*i:batch_size*(i+1)], axis=3)/255

    feed_dict = {input_tensor: img}
    out_0 = sess.run([output_tensor], feed_dict)
    
    result[batch_size*i:batch_size*(i+1)] = out_0[0]

In [22]:
print('Accuracy: %d / %d' % (np.sum(result == y_train), len(x_train)) )

NameError: name 'result' is not defined

In [25]:
help(tf.get_default_graph)

Help on function get_default_graph in module tensorflow.python.framework.ops:

get_default_graph()
    Returns the default graph for the current thread.
    
    The returned graph will be the innermost graph on which a
    `Graph.as_default()` context has been entered, or a global default
    graph if none has been explicitly created.
    
    NOTE: The default graph is a property of the current thread. If you
    create a new thread, and wish to use the default graph in that
    thread, you must explicitly add a `with g.as_default():` in that
    thread's function.
    
    Returns:
      The default `Graph` being used in the current thread.

